In [45]:
import tarfile
import os
import shutil
from astropy.io import fits
import numpy as np
from scipy.ndimage import zoom
import re
import tensorflow as tf

# Specify the directory containing the .tar files
directory_path = '/Users/matt/Dropbox/learning_neuralnets/colombialensing/'

# image_size
image_size = 256
suffix = f"_{image_size}"

# Use a regular expression to match .tar files with the desired suffix
pattern = re.compile(rf"{suffix}.tar$")

# List all matching .tar files in the directory
all_tar_files = [f for f in os.listdir(directory_path) if pattern.search(f)]

def get_labels_for_file(tar_file_name):
    """
    Extracts labels from the tar file name.
    For the file "Om0.183_si0.958_256.tar", the labels will be [0.183, 0.958].
    
    Args:
    - tar_file_name (str): Name of the tar file.
    
    Returns:
    - list: List containing the two labels extracted from the filename.
    """
    # Split the filename on underscores
    parts = tar_file_name.split('_')

    # Extract the numeric values for 'Om' and 'si'
    om_label = float(parts[0][2:])
    si_label = float(parts[1][2:])
    
    return [om_label, si_label]



def extractBatchOld(tar_file):
    tar_file_path = os.path.join(directory_path, tar_file)
    
    # Extract the tar archive
    with tarfile.open(tar_file_path, 'r') as archive:
        archive.extractall(path=directory_path)

    dir_name = os.path.splitext(tar_file)[0]
    dir_path = os.path.join(directory_path, dir_name)

    all_files = os.listdir(dir_path)
    fits_files = [f for f in all_files if f.endswith('.fits')]

    # Create a numpy array of desired shape
    data_array = np.empty((len(fits_files), image_size, image_size), dtype=np.float16)

    for idx, file in enumerate(fits_files):
        with fits.open(os.path.join(dir_path, file)) as hdul:
            
            original_data = hdul[0].data
            
            #get rid of NANs, which affects a few files
            if np.isnan(original_data).any():
                continue
                
            data_array[idx] = original_data
            
            
    #since all files have the same labels
    labels = get_labels_for_file(tar_file)
    
    batch_size = len(fits_files)
    batch_labels = np.array([labels for i in range(batch_size)])
    print(np.shape( batch_labels ))
    
    WL_labels = tf.convert_to_tensor(batch_labels)

    # At this point, data_array contains all the data from the fits files and is ready to be used with TensorFlow
    # For example, you can convert it to a TensorFlow tensor:
    WL_tensor = tf.convert_to_tensor(data_array)

    shutil.rmtree(dir_path)

    print(f"Extracted data from {tar_file}")
    return [WL_tensor, WL_labels]



def extractBatch(number_files):
    
    for tar_file in all_tar_files:
    tar_file_path = os.path.join(directory_path, tar_file)
    
    # Extract the tar archive
    with tarfile.open(tar_file_path, 'r') as archive:
        archive.extractall(path=directory_path)

    dir_name = os.path.splitext(tar_file)[0]
    dir_path = os.path.join(directory_path, dir_name)

    all_files = os.listdir(dir_path)
    fits_files = [f for f in all_files if f.endswith('.fits')]

    # Create a numpy array of desired shape
    data_array = np.empty((len(fits_files), image_size, image_size), dtype=np.float16)

    for idx, file in enumerate(fits_files):
        with fits.open(os.path.join(dir_path, file)) as hdul:
            
            original_data = hdul[0].data
            
            #get rid of NANs, which affects a few files
            if np.isnan(original_data).any():
                continue
                
            data_array[idx] = original_data
            
            
    #since all files have the same labels
    labels = get_labels_for_file(tar_file)
    
    batch_size = len(fits_files)
    batch_labels = np.array([labels for i in range(batch_size)])
    print(np.shape( batch_labels ))
    
    WL_labels = tf.convert_to_tensor(batch_labels)

    # At this point, data_array contains all the data from the fits files and is ready to be used with TensorFlow
    # For example, you can convert it to a TensorFlow tensor:
    WL_tensor = tf.convert_to_tensor(data_array)

    shutil.rmtree(dir_path)

    print(f"Extracted data from {tar_file}")
    return [WL_tensor, WL_labels]



In [39]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

 #make less redundnat

def create_cnn_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))  
    
    return model

model = create_cnn_model((image_size, image_size, 1))  # Assuming grayscale images


In [49]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [41]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 254, 254, 32)      320       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 125, 125, 64)      18496     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 62, 62, 64)       0         
 2D)                                                             
                                                                 
 flatten_4 (Flatten)         (None, 246016)            0         
                                                                 
 dense_8 (Dense)             (None, 64)               

In [50]:
count =0
for tar_file in all_tar_files:
    # ... [Code to extract and preprocess the data into data_array]

    [WL_tensor, labels]  = extractBatchold(tar_file)
    #tensor = tf.convert_to_tensor(data_array)
    
    # Assuming you have corresponding labels for each batch
    #labels = get_labels_for_batch(tar_file)  # You'd need to define this function

    print("labels shape = ", np.shape(labels))
    # Reshape tensor to be (-1, image_size, image_size, 1) to fit the CNN input shape
    WL_tensor = tf.reshape(WL_tensor, (-1, image_size, image_size, 1))

    # Train the model on this batch
    model.fit(WL_tensor, labels, epochs=5)  # You can adjust the number of epochs as needed

    print("Completed ", count/len(all_tar_files))
    count += 1


(512, 2)
Extracted data from Om0.518_si0.611_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 16s 843ms/step - loss: 0.0181 - accuracy: 0.5332
Epoch 2/5
16/16 [==============================] - 14s 845ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 12s 778ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 12s 733ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 11s 695ms/step - loss: 0.0042 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.375_si0.332_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 20s 1s/step - loss: 0.0231 - accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 20s 1s/step - loss: 0.0220 - accuracy: 0.3125
Epoch 3/5
16/16 [==============================] - 20s 1s/step - loss: 0.0217 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] 

16/16 [==============================] - 17s 1s/step - loss: 0.0431 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 17s 1s/step - loss: 0.0431 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 18s 1s/step - loss: 0.0431 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 16s 1s/step - loss: 0.0430 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 14s 867ms/step - loss: 0.0430 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.313_si0.633_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 16s 995ms/step - loss: 8.3191e-04 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 17s 1s/step - loss: 8.2727e-04 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 16s 984ms/step - loss: 8.1899e-04 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 15s 959ms/step - loss: 8.1052e-04 - accuracy: 1.0000
Epoch 5/5
16/16 [==============

16/16 [==============================] - 16s 1s/step - loss: 0.0174 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 16s 972ms/step - loss: 0.0171 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 17s 1s/step - loss: 0.0169 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 15s 909ms/step - loss: 0.0167 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.192_si0.727_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 15s 914ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 17s 1s/step - loss: 0.0046 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 17s 1s/step - loss: 0.0042 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 16s 992ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 14s 871ms/step - loss: 0.0037 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.283_si0.805

16/16 [==============================] - 15s 935ms/step - loss: 0.0034 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 15s 913ms/step - loss: 0.0032 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 14s 855ms/step - loss: 0.0030 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.191_si0.924_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 15s 971ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 17s 1s/step - loss: 0.0131 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 15s 932ms/step - loss: 0.0123 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 15s 925ms/step - loss: 0.0115 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 12s 767ms/step - loss: 0.0108 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.513_si0.256_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 15s 96

16/16 [==============================] - 14s 861ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 13s 822ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 11s 684ms/step - loss: 0.0025 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.410_si0.927_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 14s 908ms/step - loss: 0.0362 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 14s 886ms/step - loss: 0.0356 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 13s 841ms/step - loss: 0.0350 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 12s 768ms/step - loss: 0.0344 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 11s 699ms/step - loss: 0.0339 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.273_si1.204_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 14s

16/16 [==============================] - 12s 756ms/step - loss: 0.0048 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 11s 706ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 11s 667ms/step - loss: 0.0040 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.253_si0.852_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 13s 833ms/step - loss: 0.0287 - accuracy: 1.0000
Epoch 2/5
16/16 [==============================] - 13s 839ms/step - loss: 0.0270 - accuracy: 1.0000
Epoch 3/5
16/16 [==============================] - 11s 717ms/step - loss: 0.0247 - accuracy: 1.0000
Epoch 4/5
16/16 [==============================] - 11s 694ms/step - loss: 0.0226 - accuracy: 1.0000
Epoch 5/5
16/16 [==============================] - 12s 717ms/step - loss: 0.0208 - accuracy: 1.0000
(512, 2)
Extracted data from Om0.638_si0.250_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 14s

16/16 [==============================] - 15s 919ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/5
16/16 [==============================] - 13s 840ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/5
16/16 [==============================] - 13s 788ms/step - loss: nan - accuracy: 0.0000e+00
(512, 2)
Extracted data from Om0.268_si0.727_256.tar
labels shape =  (512, 2)
Epoch 1/5
16/16 [==============================] - 14s 885ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 16s 1s/step - loss: nan - accuracy: 0.0000e+00
Epoch 3/5
16/16 [==============================] - 15s 951ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 4/5
16/16 [==============================] - 16s 991ms/step - loss: nan - accuracy: 0.0000e+00
Epoch 5/5
16/16 [==============================] - 13s 826ms/step - loss: nan - accuracy: 0.0000e+00
(512, 2)
Extracted data from Om0.195_si0.994_256.tar
labels shape =  (512, 2)
Epoch 1/5
 8/16 [==============>...............] 

KeyboardInterrupt: 